# Targeted advertising with mined Twitter data
Vincent Sayseng (last update: January 3, 2020)

## Table of contents

1. [Motivation](#motivation)
2. [Clustering methods and results](#methodsresults)
    1. [Data exploration](#explore)
    2. [Text wrangling of profile descriptions](#wrangle)
    3. [Clustering users based on their profile descriptions](#clustering)
        1. [K-modes clustering](#kmodes)
        2. [DBSCAN clustering](#dbscan)
    4. [Clustering users based on their friends](#friendcluster)
3. [Business strategy recommendations](#strategy)
    1. [Recommendations for Facebook ads](#fb)
    2. [Recommendations for Twitter ads](#twitter)

## Motivation <a name='motivation'></a>

Proper audience targeting is a critical component of a successful advertising campaign. This study aims to identify customer demographics that may be particularly receptive to Game Revenant's (GR) advertising, specifically to optimize social media-based campaigns. 

Due to their large reach and popularity, Facebook, Instagram, and Twitter can be an effective means of reaching new audiences. Facebook (which owns Instagram) and Twitter both offer APIs that allow businesses to target their advertisements based on a number of demographic features, such as age, gender, language, location, and interests. This project aims to inform audience targeting based on interests. 

Analysis is performed on unstructured social media data, specifically from Twitter. Compared to other social media, Twitter's API allows greater access to user data, enabling more effective data mining. GR currently has too few followers on Twitter to allow for productive data mining. However, the Twitter accounts of rival companies can be analyzed as well. GR's game *Where Shadows Slumber* (WSS) is frequently compared to *Monument Valley*, a mobile puzzle game produced by USTWO Games, by game critics and customer reviewers. At the time of scraping, the USTWO Games Twitter account, @ustwogames, had over 126k followers. 

Customer interests were estimated by 1) examining the profile description of @ustogames followers, and 2) examining the most popular friends among @ustogames followers. Preprocessed descriptions were examined via two clustering approches, K-modes and 
DBSCAN. In Twitter's official terminology, a 'friend' is an account a user follows. The most popular friends were clustered via K-modes. The rationale behind the unorthodox application of K-modes will be explained later in the notebook. 


## Clustering methods and results <a name='methodsresults'></a>

### Data exploration <a name='explore'></a>

The Twitter profiles of @ustogames were mined via the python package Tweepy and written to a SQL database (refer to *pull_data_twitter.py* and *sqlite_fx.py* in my repository at https://github.com/vin-say/web-scraping/tree/master/twitter). 

In [1]:
import pandas as pd
import sqlalchemy as sa
from pathlib import Path

# Move to main directory of the Customer-Segmentation project
%cd ..

DB_NAME = 'customer-segmentation'
TAB_NAME = 'ustwo_followers'

# Pull data from SQLite DB
e = sa.create_engine('sqlite:///./data/interim/' + DB_NAME + '.sqlite')
query = 'SELECT * FROM ' + TAB_NAME
users = pd.read_sql_query(query, e)

users.head()

C:\Users\Vincent\Game-Revenant\Where-Shadows-Slumber\Customer-Segmentation


,id,id_string,name,screen_name,location,url,description,protected,verified,followers_count,friend_count,listed_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,1114109046792032256,1114109046792032256,Hiyoru,Hiyoru6,None,None,so para fotos de desenhos,0,0,1,21,0,139,5,Fri Apr 05 10:14:30 +0000 2019,1,0
1,1120790192128954371,1120790192128954371,Tom Baines 🏳️‍🌈 🇪🇺,TomBaines16,"North West, England",None,Extreme sports calendar model. \nKeeping retro...,0,0,29,169,0,961,456,Tue Apr 23 20:42:59 +0000 2019,0,0
2,113652889,113652889,Marmalade Games,MarmaladeGames,"London, UK",http://t.co/sPUf5LShHE,None,0,0,506,262,13,304,471,Fri Feb 12 15:07:07 +0000 2010,0,0
3,363116063,363116063,MIT SHAH,mitshah97,Ahmedabad,None,#Unity3d #Game #Developer,0,0,39,279,1,22,13,Sat Aug 27 15:13:29 +0000 2011,1,0
4,972169508,972169508,Kyrie E.H.C.,KyrieEHC,"Madison, WI",https://t.co/ajE03P8x2g,Still believes in the warmth in interaction. S...,0,0,333,1103,18,1118,734,Mon Nov 26 15:27:59 +0000 2012,0,0


In [2]:
print(str(len(users)) + ' @ustwogames follower profiles were mined')

125925 @ustwogames follower profiles were mined


### Text wrangling of profile descriptions <a name='wrangle'></a>

Although a number of features were mined, only the profile description and identifying user ID are of interest in the proceeding analysis. 

Prior to analysis, unstructured text data must be preprocessed. Punctuation, emojis, and excess white space were removed. All words were converted to lowercase and tokenized. Stopwords were removed. The NLTK stopword list was extended by several words that were found to be common in profile descriptions, but did not add any useful information about user interests. 

In [3]:
import nltk
import re
import numpy as np

def normalize_doc(doc, stop_words):
    ''' Basic text cleaning for English documents
    
    param doc (str): document to be cleaned
    param stop_words (str list): common words to be filtered out
    return clean_doc (str): cleaned document
    '''
    # remove special characters and white space. This filters out non-Latin languages!!
    doc = re.sub(r'[^a-zA-Z\s]', ' ', doc, re.IGNORECASE|re.ASCII)
    # remove single characters
    doc = re.sub(r'\b[a-zA-Z]\b', '', doc, re.IGNORECASE|re.ASCII)    
    # remove whitespace at beginning and end of string
    doc = doc.strip()
    # convert all characters to lowercase
    doc = doc.lower()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # recreate doc from filtered tokens
    clean_doc = ' '.join(filtered_tokens)
    return clean_doc

stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['co', 'https', 'http', 'gmail', 'com', 'like', 'love'])

users['clean_desc'] = users['description'].map(lambda doc: normalize_doc(doc, stop_words) 
                                                if doc is not None
                                                else np.nan)

users[['description','clean_desc']].head()

,description,clean_desc
0,so para fotos de desenhos,para fotos de desenhos
1,Extreme sports calendar model. \nKeeping retro...,extreme sports calendar model keeping retro al...
2,None,NaN
3,#Unity3d #Game #Developer,unity game developer
4,Still believes in the warmth in interaction. S...,still believes warmth interaction studied game...


The results after initial preprocessing are shown above. For now, only profile descriptions in English were analyzed. Language identification of short text is challenging, and the more common language identication libraries (e.g. langid) were inaccurately classifying certain profile descriptions. Facebook's FastText library offers an alternative that has shown relatively high accuracy with short text language classification (http://alexott.blogspot.com/2017/10/evaluating-fasttexts-models-for.html). 

English profiles were lemmatized using the NLP libary spaCy, completing the text preprocessing stage of the analysis pipeline. Lemmatization can take several minutes, so the updated DataFrame was saved to a csv file. 

In [4]:
import os
import fasttext
import spacy

nlp = spacy.load('en') #download 'small' version of english model

def lemmatize_doc(doc):
    '''Apply spaCy lemmatization to a document (str)'''
    
    doc = nlp(doc)
    doc = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in doc])
    return doc

# import and apply FastText model for language identification
idlang_path = 'data/external/fasttext_training_data/lid.176.bin'
idlang_model = fasttext.FastText.load_model(idlang_path)

users['lang_id'] = users['clean_desc'].map(lambda doc: idlang_model.predict(doc) 
                                            if doc == doc 
                                            else np.nan)

# only English profiles are lemmatized; empty profiles are also ignored
users['clean_desc_en'] = users.apply(lambda row: lemmatize_doc(row['clean_desc']) 
                                        if  row['lang_id'] == row['lang_id']
                                        and row['lang_id'][0][0] == '__label__en'
                                        and row['clean_desc'] == row['clean_desc'] 
                                        else np.nan,
                                        axis=1)

# Push processed data into SQLite DB. Profiles with non-English or empty descriptions were dropped
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '-clean.sqlite')

cleaned_tab = TAB_NAME + '_clean'
users[['id', 'description', 'clean_desc_en']].to_sql(cleaned_tab, e, if_exists='replace')

users[['description','clean_desc','clean_desc_en']].head()

,description,clean_desc,clean_desc_en
0,so para fotos de desenhos,para fotos de desenhos,NaN
1,Extreme sports calendar model. \nKeeping retro...,extreme sports calendar model keeping retro al...,extreme sport calendar model keep retro alive ...
2,None,NaN,NaN
3,#Unity3d #Game #Developer,unity game developer,unity game developer
4,Still believes in the warmth in interaction. S...,still believes warmth interaction studied game...,still believe warmth interaction study game cu...


### Clustering users based on their profile descriptions <a name='clustering'></a>

After preprocessing, the profile descriptions were clustered via two approaches, K-modes and DBSCAN. 

#### K-modes clustering <a name='kmodes'></a>

Given the short length of a typical profile, a word will typically appear no more than once within a given profile description. *It would thus be more approriate to vectorize profile descriptions as binary categorical vectors, as opposed to numeric vectors.* Profile descriptions were count vectorized as binary categorical data, where a value of *1* would indicate that a given word was present in a given profile description.  


K-means is likely the most popular approach to clustering. However, its use is limited to numerical data. The K-modes algorithim was developed as an analog to K-means for categorical data (Huang 1997, 1998). Like K-means, the K-modes algorithim requires that the number of clusters be predetermined.  

The kmodes Python library was used to perform K-modes clustering. No optimal number of clusters was assumed a priori. Consequently, K-modes clustering for a a range of initial cluster numbers (2 through 20) was executed. Results were saved to a SQL database. 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from kmodes.kmodes import KModes

DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'
NUM_CLUSTERS_RNG = 16 #upper limit of cluster range to evaluate

# Pull processed data from SQLite DB
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')

#SQL query includes a WHERE statement that filters out NA values and empty strings
query = 'SELECT id, description, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
users = pd.read_sql_query(query, e)

cv = CountVectorizer(ngram_range=(1, 2), min_df=100, max_df=1.0,
                     stop_words=stop_words, binary=True)
cat_matrix = cv.fit_transform(users['clean_desc_en'])

# Connect to DB where cluster results are to be saved
DB_RESULTS_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
TAB_RESULTS_PREFIX = 'ustwo_followers_k_'

for n_clusters in range(2, NUM_CLUSTERS_RNG):
    kmod = KModes(n_clusters=n_clusters, init='Huang', random_state=42, n_jobs=-1)
    y_pred = kmod.fit_predict(cat_matrix.toarray())
    users['cluster'] = kmod.labels_
    tab_name = TAB_RESULTS_PREFIX + str(n_clusters)
    users.to_sql(tab_name, e, if_exists='replace')
    print('k = ' + str(n_clusters) + ' clusters analyzed')

k = 2 clusters analyzed
k = 3 clusters analyzed
k = 4 clusters analyzed
k = 5 clusters analyzed
k = 6 clusters analyzed
k = 7 clusters analyzed
k = 8 clusters analyzed
k = 9 clusters analyzed
k = 10 clusters analyzed
k = 11 clusters analyzed
k = 12 clusters analyzed
k = 13 clusters analyzed
k = 14 clusters analyzed
k = 15 clusters analyzed


The Kmodes algorithim will force the data to be organized into the predefined number of clusters. Clusters are not guaranteed to be generated in a meaningful way. 

In order to gauge the distinctiveness of the clusters, a vocabulary list of the most common unigrams, bigrams, and trigrams within the profile descriptions of a given cluster are generated (*gen_vocab_list*) and ordered by freqeuncy of occurance. The degree of similiarity in the top vocabulary between two clusters is a measure of the of the distance between those clusters.

In [6]:
from collections import Counter

def gen_vocab_list(corpus):
    ''' Generates a vocab list of unigrams, bigrams, and trigrams found in the corpus
    
    param corpus (str list): list containing documents from which vocab list will be derived
    return vocab (str list): list of vocab words found in the corpus
    '''
    unigrams = [words for doc in corpus for words in doc.split()]
    ngrams = [bigram for doc in corpus for bigram in nltk.ngrams(doc.split(), 2)]
    bigrams = [token[0] + ' ' + token[1] for token in ngrams]
    ngrams = [bigram for doc in corpus for bigram in nltk.ngrams(doc.split(), 3)]
    trigrams = [token[0] + ' ' + token[1] + ' ' + token[2] for token in ngrams]
    vocab = unigrams + bigrams + trigrams
    return vocab

def print_cluster_results(df, col_clust, col_list, clust_rng, n_keywords=10, n_desc=5):
    '''Evaluate clustering results by printing the most popular vocab words within a cluster corpus, and the prevalence of those words in the cluster corpus 
    
    param df (DataFrame): contains columns with profile features and cluster assignment
    param col_clust (str): name of column containing cluster assignment
    param col_list (str list): name of column containing corpus to be analyzed
    param clust_rng (tuple): bounds of the range of clusters to be printed. Assumes integer increments of 1. 
    param n_keywords (int): number of top vocab words to be analyzed for prevalence in the corpus
    param n_desc (int): number of randomly sampled twitter profiles whose features listed in col_list are to be printed
    '''
    n_total = len(df) 
    
    for cluster in range(clust_rng[0], clust_rng[1]):
        corpus = df.loc[df[col_clust] == cluster]
        n_corpus = len(corpus)
        per_label = round(n_corpus/n_total*100) # percentage of documents assigned to the cluster label 
        vocab = gen_vocab_list(corpus[col_list[0]])
        print('Cluster # ' + str(cluster))
        print(str(n_corpus) + ' (' + str(per_label) + '%) users with valid description in this cluster')
        print(Counter(vocab).most_common(n_keywords))
        if n_desc > 0:
            print(corpus[col_list].sample(n=n_desc))
        print('\n')

All the cluster results can then be analyzed, pulling the K-modes output from the database.

In [7]:
DB_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')

for n_clusters in range(2, NUM_CLUSTERS_RNG):

    TAB_NAME = 'ustwo_followers_k_' + str(n_clusters)
    query = 'SELECT id, clean_desc_en, cluster FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
    users = pd.read_sql_query(query, e)

    print('CLUSTER K = ' + str(n_clusters))
    print('\n')
    print_cluster_results(df=users, col_clust='cluster', col_list=['clean_desc_en'], clust_rng=(0,n_clusters))

CLUSTER K = 2


Cluster # 0
47166 (98%) users with valid description in this cluster
[('game', 10080), ('designer', 3933), ('make', 2676), ('developer', 2441), ('design', 2342), ('gamer', 2325), ('artist', 2257), ('video', 2101), ('play', 2036), ('follow', 1895)]
                                           clean_desc_en
35439  quality engineer slackhq ex flickr photography...
13614                             mobile game io android
38072                        life short craft long learn
45946  writer student game player always order write ...
45428  interactioneer ceo vl ltb zvs make wuppet wupp...


Cluster # 1
1058 (2%) users with valid description in this cluster
[('creative', 1092), ('director', 335), ('creative director', 271), ('designer', 198), ('game', 189), ('design', 172), ('digital', 107), ('founder', 91), ('art', 90), ('studio', 76)]
                                           clean_desc_en
6124   founder creative art director brokenrule game ...
16527  design creative speak

10124  game designer focus narrative ai garbage anime...


CLUSTER K = 6


Cluster # 0
43812 (91%) users with valid description in this cluster
[('game', 9153), ('designer', 3500), ('make', 2472), ('developer', 2313), ('gamer', 2217), ('artist', 2088), ('video', 1961), ('play', 1937), ('work', 1666), ('follow', 1644)]
                                           clean_desc_en
26631                                        turn nation
15353            unity developer focus mobile game vr ar
10535              story inspiration behind maker admire
35142  political science santannapisa intern antitrus...
1184   super intuitive retro revival mobile game whol...


Cluster # 1
1095 (2%) users with valid description in this cluster
[('director', 1137), ('creative', 328), ('art', 288), ('creative director', 269), ('game', 241), ('art director', 237), ('designer', 200), ('design', 173), ('founder', 139), ('digital', 86)]
                                           clean_desc_en
35649                

37826  mazlynify gaming channel youtube enjoy play vi...


Cluster # 1
7183 (15%) users with valid description in this cluster
[('game', 8563), ('developer', 1081), ('designer', 1006), ('make', 925), ('game developer', 760), ('play', 748), ('indie', 736), ('artist', 622), ('game designer', 605), ('work', 590)]
                                           clean_desc_en
3320   indie game dev make golf fun join party lif bd...
18284                                       game awesome
17227  professional gamer game developer wait mention...
26565  brand new youtube channel everything cod gbs p...
45692  independent solo game dev contact need music a...


Cluster # 2
1297 (3%) users with valid description in this cluster
[('developer', 1333), ('web', 220), ('software', 190), ('designer', 183), ('io', 157), ('web developer', 145), ('software developer', 144), ('io developer', 98), ('end', 90), ('app', 89)]
                                           clean_desc_en
46221                        fro

2587   freelance designer writer mixtape guru master ...


Cluster # 7
156 (0%) users with valid description in this cluster
[('game', 187), ('player', 161), ('game player', 26), ('developer', 24), ('play', 23), ('player game', 19), ('designer', 17), ('game developer', 16), ('make', 14), ('mobile', 12)]
                                           clean_desc_en
27902               game hate player hate game soldy non
20043  illustrator game player friendly guy general c...
37173  founder ceo wizit game studio horrible bass gu...
45560  game player ex maker swimrunner sneakerwearer ...
46022                  game player game scholar lecturer


Cluster # 8
5196 (11%) users with valid description in this cluster
[('game', 6036), ('developer', 819), ('play', 611), ('game developer', 608), ('indie', 573), ('indie game', 443), ('artist', 417), ('dev', 386), ('work', 382), ('studio', 369)]
                                           clean_desc_en
42015  husband father author principal thebiggerd

2871   frontend wingify typescript fan indie game dev...


Cluster # 2
371 (1%) users with valid description in this cluster
[('see', 386), ('game', 74), ('make', 31), ('follow', 28), ('video', 28), ('guy', 28), ('thing', 27), ('new', 27), ('world', 25), ('life', 23)]
                                           clean_desc_en
19419  guy laptrap make youtube video make gaming vol...
44347  one direction life happen see wish would messa...
46799  contributor fakeapstylebook kevin church profe...
26864  choose unseen know people want see sometimes h...
28029  teen step life style loop get see age kik neon...


Cluster # 3
1687 (3%) users with valid description in this cluster
[('art', 1790), ('game', 593), ('design', 307), ('director', 283), ('artist', 236), ('designer', 236), ('art director', 236), ('music', 153), ('make', 134), ('game art', 122)]
                                           clean_desc_en
45505                       strategy design art bike dad
44926  illustrator art directo

1726 (4%) users with valid description in this cluster
[('work', 1783), ('game', 777), ('currently', 271), ('designer', 266), ('currently work', 233), ('developer', 202), ('artist', 196), ('make', 155), ('design', 118), ('studio', 116)]
                                           clean_desc_en
5088    pen ink artist work indie game land eyas qfyn st
3478   game artist sl uninorthant ex aaa indie game w...
43711  game designer musician magician human currentl...
46237  two creative lukewicker iamwilf work part fact...
41985  happy engineer intellimob net umbraco azure si...


Cluster # 6
515 (1%) users with valid description in this cluster
[('try', 522), ('game', 125), ('make', 114), ('try make', 76), ('get', 72), ('play', 45), ('try get', 45), ('life', 43), ('thing', 42), ('video', 42)]
                                           clean_desc_en
8633   guy try often possible make food game bud ui g...
120    illustration hobbyist predominantly traditiona...
10571  sell custom one vr heads

29310  una tal dulcinea cofounder design build studio...


Cluster # 7
1141 (2%) users with valid description in this cluster
[('gaming', 1204), ('youtube', 206), ('channel', 138), ('video', 106), ('gamer', 102), ('follow', 102), ('xbox', 76), ('check', 75), ('subscribe', 71), ('clan', 65)]
                                           clean_desc_en
30731  host multiple gaming tournament every month la...
12295  vegan gaming switch atm proud hufflepuff lingu...
1765   wish mom would get fact bear gaming would buy ...
38420  glitchforum online general gaming computing co...
17545                                gaming organization


Cluster # 8
146 (0%) users with valid description in this cluster
[('mine', 155), ('opinion', 47), ('view', 39), ('opinion mine', 38), ('view mine', 29), ('tweet', 22), ('alone', 14), ('mine alone', 14), ('tweet mine', 14), ('designer', 12)]
                                           clean_desc_en
39300  ui ux specialist pixel artist photography nerd...
13886  d

551 (1%) users with valid description in this cluster
[('channel', 590), ('youtube', 582), ('youtube channel', 490), ('check', 155), ('subscribe', 138), ('gaming', 103), ('game', 91), ('check youtube', 88), ('check youtube channel', 86), ('subscribe youtube', 67)]
                                           clean_desc_en
12618     crapy youtube channel want make gaming channel
24555  hey guy noah speak go youtube channel noah upl...
27056       check youtube channel please subscribe thank
21665               twitt get hit gaming youtube channel
34148                                    youtube channel


Cluster # 9
350 (1%) users with valid description in this cluster
[('man', 368), ('game', 53), ('one', 30), ('family', 28), ('family man', 25), ('developer', 19), ('life', 18), ('fan', 18), ('one man', 17), ('gamer', 16)]
                                           clean_desc_en
3185   ridiculous man responsible song summer sorry t...
18670                                           good ma

[('designer', 2857), ('graphic', 467), ('ux', 409), ('graphic designer', 396), ('product', 307), ('design', 284), ('illustrator', 281), ('product designer', 249), ('ui', 228), ('ux designer', 228)]
                                           clean_desc_en
3647                               ux designer leet tech
16436                                    designer planet
44680  human overall product designer facebook former...
12941  surrey bear motion graphic designer love earl ...
10920              art director ux ui designer hcsf umbz


Cluster # 9
232 (0%) users with valid description in this cluster
[('london', 245), ('base', 54), ('design', 43), ('digital', 32), ('director', 32), ('base london', 28), ('art', 25), ('creative', 24), ('london base', 23), ('artist', 18)]
                                           clean_desc_en
3628   architect collcoll work eva jiricna architect ...
11084  award win zero emission network help business ...
2661   official page rauli rd art gizmo riihim ki

Surveying the cluster results, common themes consistently arise. Gaming is clearly the most common interest among @ustwogames Twitter followers. Professions associated with the industry are also fairly popular, e.g. game designers, graphic designers, and UX designers. A few notable examples include the following:
 - *K=8, Cluster #3*: video game enthusiasts that stream on YouTube
 - *K=9, Cluster #8*: game developers, particularly indie studios
 - *K=10, Cluster #9*: creatives, e.g. graphic designers, artists, illustrators.

Of course, not all clusters are informative. For example, *K=6, Cluster #4* is primarily characterized by descriptions that include the word "hi". 

#### DBSCAN clustering <a name='dbscan'></a>

DBSCAN (density-based spatial clustering of applications with noise) is a density-based clustering algorithim. Unlike K-modes, which will force each data point to be assigned to a cluster, DBSCAN will mark data in low-density regions as outliers. 

Descriptions were vectorized by normalized term frequency. 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'

# Pull processed data from SQLite DB
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')

#SQL query includes a WHERE statement that filters out NA values and empty strings
query = 'SELECT id, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
users = pd.read_sql_query(query, e)
N_DESC = 10
N_KEYWORDS = 10

tf = TfidfVectorizer(use_idf=False)
tf_matrix = tf.fit_transform(users['clean_desc_en']) #tf is normalized as opposed to cv

dbscan = DBSCAN(eps=1.2, min_samples=1000)

y_pred = dbscan.fit_predict(tf_matrix)
users['cluster'] = dbscan.labels_

# Push processed data into SQLite DB. Profiles with non-English or empty descriptions were dropped
DB_RESULTS_NAME = 'customer-segmentation-cluster'
TAB_RESULTS_NAME = 'ustwo_followers_dbscan'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
users.to_sql(TAB_RESULTS_NAME, e, if_exists='replace')

# Print results
n_clusters = len(np.unique(dbscan.labels_))
clust_rng = (-1, n_clusters-1) # offset by -1 since DBSCAN's first cluster is #-1
print_cluster_results(df=users, col_clust='cluster', col_list=['clean_desc_en'], clust_rng=clust_rng)

Cluster # -1
14653 (30%) users with valid description in this cluster
[('world', 391), ('founder', 381), ('work', 380), ('tweet', 337), ('get', 328), ('new', 327), ('fan', 316), ('tech', 315), ('team', 303), ('good', 299)]
                                           clean_desc_en
2620                      pok mon go player shiny hunter
22901  formerly know alias cl ud twitch mod launder boxr
36567  player listener griller writer drinker travele...
46186  partner equalexpert instructor snowcentre drin...
15360  find tutor make easy stutor list tutor right a...


Cluster # 0
33571 (70%) users with valid description in this cluster
[('game', 10230), ('designer', 4115), ('make', 2613), ('developer', 2461), ('design', 2353), ('gamer', 2216), ('artist', 2185), ('video', 2092), ('play', 1971), ('follow', 1763)]
                                           clean_desc_en
1203   indie game studio base london building trading...
31796                          designer zaraguza digital
764    creator

DBSCAN determined that 70% of the cleaned Twitter descriptions were similiar enough to form a distint cluster (label = 0), while the remaining 30% were classified as outliers (label = -1). None of the top ten vocabulary words were shared between the Cluster #0 and "Cluster" #-1, suggesting good separation of the classes. 

The results corroborate those found via K-modes clustering. Based on the top vocabulary words of Cluster #0, a significant portion of USTWO Games Twitter followers are interested in gaming. This population seems to include video game fans, as well as professional or hobbyist creators: roughly of the top ten vocabulary words are associated with video game design, art, and development. 

### Clustering users based on their friends <a name='friendcluster'></a>

Clustering based on profile description has allowed us to identify useful customer segments: gamers, video game creators, developers, and Twitch/YouTube streamers. However, this approach has few a limitations. Profile descriptions are not mandatory, and many Twitter users choose to leave the field blank. Profiles that do contain a description may not contain relevant or useful information. Finally, this approach is biased towards professionals, or individuals with products or services to sell or advertise: these users have an incentive to create a profile description that is complete, informative, and accurate. 

Another approach to customer segmentation is by clustering Twitter users based on the accounts that they follow (termed 'friends'). Presumably, users will only follow friends whose content they want to see: a user's friends are a reflection of their interests.

The user ID of the friends of @ustwogames followers were collected. K-modes categorical clustering was performed to determine if users could be clustered based on the accounts they followed. 

Due to the rate limitations of the Twitter API, collecting the friend IDs of every USTO Games follower would be excessively time consuming. Stratified sampling based on the DBSCAN clustering results was performed to reduce the sample size to 10% (12592 users). 

In [9]:
# Get profiles that were clustered
DB_NAME = 'customer-segmentation-cluster'
TAB_NAME = 'ustwo_followers_dbscan'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')
query = 'SELECT id, clean_desc_en, cluster FROM ' + TAB_NAME
users_clustered = pd.read_sql_query(query, e)

# Get profiles that were not eligible for clustering (empty or not English)
DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')
query = 'SELECT id, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en == \'\' OR clean_desc_en IS NULL'
users_empty = pd.read_sql_query(query, e)
users_empty['cluster'] = -2 # create new 'cluster' assignment, to which all empty profiles are assigned

# Combine dataframes
users = users_clustered.append(users_empty, ignore_index=True)

# Stratified sampling profiles by cluster assignment, pushing results to database
TAB_SAMPLES_NAME = 'ustwo_followers_sampled'
sampled_users = users.groupby('cluster').apply(lambda x: x.sample(frac=0.1, random_state=42))
sampled_users.reset_index(drop=True).to_sql(TAB_SAMPLES_NAME, e, if_exists='replace')

The Twitter API allows scraping of the IDs of accounts a given user follows. However, if a screenname or additional profile information is desired, that would require a separate request. 

Scraping the friend IDs of the sampled @ustwogames followers yielded a table entitled *ustwo_sampled_friends_ids*. The two column table consisted of a user ID and a friend ID, i.e. one user-friend pair for each account a given user followed. 

Since scraping the profile information of every account followed by @ustwo games followers would be resource-intensive and minimally useful, only the most common friend IDs were scraped for their profile information (specifically friend IDs that had at least 75 @ustwo followers following it). This generated a table entitled *ustwo_sampled_fol_friends.*

In order to continue with K-modes, it was necessary to generate a two-column table consisting of a user ID and a list containing the screen names of every account that user followed.

This table was generated using the SQL query written below. Step-by-step, the query performs the following functions: 
1. Filter out redundant user-friend ID pairs and exclude the friend ID @ustwogames, since all users follow that account
2. Filter out friend IDs that have less than 75 users following that account, since very unpopular accounts will not be relevant when clustering
3. Join the resulting subtable containing the filtered user and friend IDs with another table (*ustwo_sampled_fol_friends*) that contains the screennames associated with a given friend ID
4. That resulting subtable was grouped by user ID, generating a second column consisting of concatenated screenames of friends

In [11]:
DB_NAME = 'customer-segmentation'
e = sa.create_engine('sqlite:///./data/interim/' + DB_NAME + '.sqlite')

query = '''
-- returns a list of friends (by screen name) that are followed by a given user
SELECT 
	user_id, GROUP_CONCAT(screen_name, " ") AS friends_list
FROM (
	-- only interested in friends that are followed by at least 75 users
	SELECT 
		user_id, friend_id
	FROM 
		ustwo_sampled_friends_ids
	WHERE
		friend_id
	IN (
		SELECT 
			friend_id
		FROM (
			-- filter out redundant user-friend id pairs
			SELECT DISTINCT
				user_id, friend_id
			FROM 
				ustwo_sampled_friends_ids
			-- filter out USTWO user id, since it will have no value in clustering since everyone follows the account
			WHERE 
				friend_id != 899902687
		)
		GROUP BY 
			friend_id
		HAVING
			COUNT(user_id) >= 75
	)
) AS tab
LEFT JOIN
	ustwo_sampled_fol_friends
ON 
	tab.friend_id == ustwo_sampled_fol_friends.id
GROUP BY 
	user_id
'''

users = pd.read_sql_query(query, e)

An example of the resulting output is shown below.

In [12]:
users.sample(n=5, random_state=42)

,user_id,friends_list
3128,1.394420e+09,briecode quasimondo lorenschmidt jesseringrose...
11176,4.751143e+09,ThomasSanders shenanigansen vaughnpinpin paper...
200,1.425125e+07,Kotaku rockpapershot OnePerfectShot FinjiCo Wh...
9480,3.285014e+09,Roblox Taymastar levine pgbiz RockstarGames gi...
8235,3.178516e+09,Twitch Pentadact TheEllenShow AnnaKendrick47 f...


K-modes clustering can now be performed. As before, no optimal K value was assumed a priori, so a range of (2 to 4) was tested. 

In [17]:
cv = CountVectorizer(min_df=1, max_df=1.0, binary=True)
cat_matrix = cv.fit_transform(users['friends_list'])

NUM_CLUSTERS_RNG = 5

# Connect to DB where cluster results are to be saved
DB_RESULTS_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
TAB_RESULTS_PREFIX = 'ustwo_fol_friends_k_'

for n_clusters in range(2, NUM_CLUSTERS_RNG):
    kmod = KModes(n_clusters=n_clusters, init='Huang', random_state=42, n_jobs=-1)
    y_pred = kmod.fit_predict(cat_matrix.toarray())
    users['cluster'] = kmod.labels_
    tab_name = TAB_RESULTS_PREFIX + str(n_clusters)
    users.to_sql(tab_name, e, if_exists='replace')
    

In [14]:

DB_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')

for n_clusters in range(2, NUM_CLUSTERS_RNG):

    TAB_NAME = 'ustwo_fol_friends_k_' + str(n_clusters)
    query = 'SELECT user_id, friends_list, cluster FROM ' + TAB_NAME
    users = pd.read_sql_query(query, e)

    print('CLUSTER K = ' + str(n_clusters))
    print('\n')
    print_cluster_results(df=users, col_clust='cluster', col_list=['friends_list'],
                          clust_rng=(0,n_clusters), n_keywords=50, n_desc=0)

CLUSTER K = 2


Cluster # 0
6292 (53%) users with valid description in this cluster
[('RockstarGames', 5627), ('Ubisoft', 5593), ('PlayStation', 5568), ('Xbox', 5567), ('IGN', 5567), ('Kotaku', 5552), ('GameSpot', 5544), ('Twitch', 5535), ('TimOfLegend', 5533), ('giantbomb', 5532), ('engadgetgaming', 5529), ('HIDEO_KOJIMA_EN', 5507), ('NintendoAmerica', 5500), ('notch', 5494), ('Polygon', 5489), ('GamesRadar', 5488), ('gamasutra', 5488), ('SupergiantGames', 5483), ('PocketGamer', 5481), ('patrickklepek', 5476), ('fullbright', 5472), ('ID_AA_Carmack', 5465), ('telltalegames', 5460), ('2K', 5449), ('EAMobile', 5435), ('pgbiz', 5423), ('GooglePlay', 5422), ('GamesBeat', 5422), ('AppStore', 5404), ('leighalexander', 5401), ('acarboni', 5394), ('Quinns108', 5392), ('Pentadact', 5392), ('WH1SKI', 5390), ('levine', 5382), ('popcap', 5352), ('bfod', 5349), ('MaxTemkin', 5282), ('tinybop', 4542), ('tha_rami', 3868), ('brandonnn', 3825), ('chrisremo', 3816), ('br', 3778), ('stephentotilo', 3777)

[('BarackObama', 1701), ('elonmusk', 1588), ('NASA', 1299), ('PlayStation', 1278), ('BillGates', 1168), ('Google', 1154), ('RockstarGames', 1124), ('Twitter', 1111), ('YouTube', 1072), ('steam_games', 1067), ('Ubisoft', 989), ('NintendoAmerica', 974), ('IGN', 952), ('Xbox', 945), ('WIRED', 914), ('tim_cook', 910), ('neiltyson', 896), ('AppStore', 886), ('SpaceX', 883), ('HIDEO_KOJIMA_EN', 881), ('Polygon', 850), ('GooglePlay', 806), ('nytimes', 800), ('Snowden', 795), ('Twitch', 793), ('TechCrunch', 790), ('verge', 773), ('Android', 770), ('Kotaku', 769), ('TheEllenShow', 755), ('gamasutra', 753), ('instagram', 751), ('POTUS44', 742), ('tha_rami', 728), ('jimmyfallon', 728), ('TimOfLegend', 712), ('unity3d', 710), ('notch', 708), ('Dropbox', 673), ('jk_rowling', 670), ('GameSpot', 665), ('EmmaWatson', 665), ('Microsoft', 665), ('RobertDowneyJr', 657), ('EA', 657), ('Medium', 656), ('Naughty_Dog', 656), ('SupergiantGames', 655), ('ustwo', 652), ('TEDTalks', 651)]


Cluster # 3
2378 (20%

Reviewing the clustering results, it is apparent that K = 3 generated to most interesting clusters. The results are (again) presented below. 

In [16]:
# %% K-modes clustering, show top keywords in each cluster

NUM_CLUSTER = 3
N_KEYWORDS = 20
N_DESC = 0

PATH = os.path.normpath('c:\\Users\\Vincent\\Game-Revenant\\Shadows\\ustwo_sampled_friend_fol_cluster')
FILE_PREFIX = 'kmode_idlimit-75_'
FULL_PATH = os.path.join(PATH, FILE_PREFIX + str(NUM_CLUSTER) + '.csv')
results = pd.read_csv(FULL_PATH).dropna()
n_total = len(results)

for cluster in range(0, NUM_CLUSTER):
    corpus = results.loc[results['cluster'] == cluster]
    n_corpus = len(corpus)
    vocab = gen_vocab_list(corpus['friend_name'])
    print('Cluster: ' + str(cluster))
    print(str(n_corpus) + ' (' + str(round(n_corpus/n_total*100)) + '%) users with valid description in this cluster')
    print(Counter(vocab).most_common(N_KEYWORDS))
    print('\n')

Cluster: 0
4500 (41%) users with valid description in this cluster
[('BarackObama', 1708), ('elonmusk', 1590), ('NASA', 1299), ('PlayStation', 1276), ('BillGates', 1170), ('RockstarGames', 1132), ('Twitter', 1112), ('steam_games', 1073), ('YouTube', 1072), ('Ubisoft', 998), ('NintendoAmerica', 979), ('IGN', 953), ('Xbox', 946), ('HIDEO_KOJIMA_EN', 889), ('AppStore', 886), ('Polygon', 860), ('GooglePlay', 813), ('Twitch', 791), ('Kotaku', 788), ('TheEllenShow', 769)]


Cluster: 1
3149 (29%) users with valid description in this cluster
[('chrisremo', 2928), ('tha_rami', 2924), ('br', 2910), ('stephentotilo', 2908), ('kirkhamilton', 2904), ('helvetica', 2903), ('ibogost', 2901), ('flantz', 2899), ('brandonnn', 2898), ('idlethumbs', 2883), ('avantgame', 2882), ('toucharcade', 2877), ('RockstarGames', 2875), ('amy_hennig', 2871), ('IGN', 2867), ('TimOfLegend', 2863), ('patrickklepek', 2863), ('femfreq', 2862), ('Ubisoft', 2860), ('Kotaku', 2859)]


Cluster: 2
3371 (31%) users with valid des

The overlap in the top 50 accounts between clusters was minimal, suggesting good separation. 

Cluster \#2 is comprised of users that are interested in mainstream gaming, as fans or consumers. Top friends include large game publishers (Rockstar Games, Ubisoft), gaming consoles (Xbox, Playstation), gaming content mediums (Twitch), and publications (IGN, Polygon). 

Cluster \#1 is composed of users that are interested in indie video games (i.e. smallar video game creators). The accounts listed are primarily associated with individual video game developers or creators. Consultation with a domain expert (Frank DiCola, founder/CEO of Game Revenant and lead designer of WSS) revealed that these users tend to be very active on Twitter, and will post content that is not strictly related to gaming (e.g. social issues, politics). 

The accounts in Cluster \#0 do not have a clear theme. A number of accounts tend to be generally popular, and have a very high number of followers (e.g. Barack Obama, Elon Musk). Another indication is that ratio between the users that follow the top accounts and the total number of users in the cluster is low (<0.5). 

## Business strategy recommendations <a name='strategy'></a>

The following demographics should be prioritized in ad targeting

 - Video game fans and enthusiasts
 - Video game professionals
 - Indie game fans and professionals
 
Facebook and Twitter differ in their ad targeting capabilities. Recommendations will be specific to what each platform offers. 

### Recommendations for Facebook ads <a name='fb'></a>

Facebook allows targeting based on it users' profession and interests. Facebook-specific recommendations are as follows:
1. **Gaming professionals**. Users can be targeted based on their profession. Based on the K-modes analysis of @ustogames follower descriptions, professionals such as game designers, graphic designers, UX designers, illustrators and other creatives should be specifically targeted. 
2. **"Mainstream" video game fans**. Based on Cluster \#2 of the DBSCAN analysis of @ustogames follower friends, Facebook users who have expressed interest in popular gaming studios (Ubisoft, Rockstar Games), platforms (Xbox, Playstation), and media (Gamespot, Twitch) should be specifically targeted. 
3. **Indie video game fans**. Based on clustering analysis on @ustogames follower descriptions and friends, smaller independent gaming studios, developers, and media should specifically be targted.

A/B testing can be implemented to determine if any particular demographic is especially receptive to WSS advertising. 

### Recommendations for Twitter ads <a name='twitter'></a>

Follower targeting is available on Twitter. Twitter users similiar to the followers of a list of usernames will be targeted. From the results of the DBSCAN analysis of @ustwo follower friends, two user name lists can be generated, one for the usernames in Cluster \#1 and another for those in Cluster \#2. 

Follower targeting can also be performed with @ustogames and other similiar mobile premium game Twitter accounts.

Finally, keyword targeting is also available on Twitter. The keywords recommended for Facebook targeting can be used similarily. 